# <center>Lecture 4. Graphical Methods of analyzing ODEs
## <center>Denis Titov
## <center>January 29, 2021

**The goal of this jupyter notebook is to introduce you to graphical methods of analyzing differential equations without solving them. We will also review various options for plotting data using python library matplotlib.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

## Introduction to Graphical Methods

Let's take an ODE $\frac{dy(t)}{dt}=f(t,y(t))$

It is possible to plot the outlines of this ODE solution using only $f(t,y(t))$ without solving the differential equation.  
Previously we reviewed numerical methods of solving ODE that can give us the solution of this ODE starting from some initial conditions $y(t_0)$. However, there's only one solution of this ODE that goes through $y(t_0)$ and there's an infinite number of other solutions. In addition, sometime there's no solution that goes through $y(t_0)$.  
  
  Therefore, it is often useful to be able to "see" how all the solutions look like using graphical methods reviewed here even though we can't use graphical methods to make precise quantitative prediction like with numerical or analytical solutions.  
Some the properties of solutions that we can see using graphical methods include steady state, oscillation, stable and unstable points etc.




Let's use bacterial growth rate in rich media as a first example:
  
$\frac{dN(t)}{dt}=kN(t)$  

where $t$ is time,  
k is some constant specific for this E.coli strain and growth conditions,  
$N(t)$ is number of E. coli at time $t$.  
Let's plot a numerical solution to this ODE with $N(0)=1$, $k=1$ and lets make this plot more beautiful than previously:

In [ ]:
k = 1
N_0 = [1]
tspan = [0, 5]

def fun(t, y):
    dN_dt = k * y
    return dN_dt

soln = solve_ivp(fun, tspan, N_0, t_eval=np.linspace(0, 5, 30))

# The code below plots the solution of ODE
plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0], label="$N(t)=N_0e^{kt}$", color="green", marker="o",linestyle="-")
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("$N(t)$", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=20)
plt.savefig("beautiful_figure.png")

We want to know how the solution will look like everywhere not just at $N(0)=1$.  
  
  Remember that the anlytical solution to $\frac{dN(t)}{dt}=kN(t)$ is $N(t)=Ce^{kt}$ so there's an $\infty$ of solutions that vary by $C$.  
  
  Lets review another example $\frac{dy(t)}{dt}=2t$ and its solution $y(t)=t^2+C$ to further highlight the presence of an $\infty$ of solutions that vary by $C$:

$\frac{dy(t)}{dt}=\lim_{\Delta t \to 0} \frac{y(t+\Delta t)-y(t)}{\Delta t}$  
  
  $\frac{y(t+\Delta t)-y(t)}{\Delta t}=\frac{(t+\Delta t)^2+C-(t)^2-C}{\Delta t}=\frac{t^2+2t \cdot \Delta t + \Delta t^2-t^2}{\Delta t}=2t + \Delta t$  
  
  Since $\Delta t \to 0$, we get $\frac{dy(t)}{dt}=2t$


We can of course numerically solve this ODE under 1000s of initial conditions but this can take time and we still might miss something important like oscillations occuring only with particular initial conditions.  

## Direction Fields (also called Slope Fields)

We can get a general idea of the behaviour of ODE or system of ODEs by drawing a so called Direction Field or Slope Field.  
First, let's review that a differential $\frac{dy(t)}{dt}$ at a point $(t, y(t))$ in ODE $\frac{dy(t)}{dt}=f(t,y(t))$ is line tangent with slope $f(t,y(t))$.  
Let's plot this tangent for our example equation $\frac{dN(t)}{dt}=kN(t)$:

In [ ]:
time = np.linspace(0, 5, 100)
OD600 = np.exp(time)
i=80
a=OD600[i]
b=(1-time[i])*OD600[i]

plt.figure(figsize=(8,5),dpi=100)
plt.plot(time, OD600, label="$N(t)=N_0e^{kt}$")
plt.plot(time,a*(time)+b, label=f"Tangent line at N(t)={OD600[i]:.2f}, t={time[i]:.2f}")
plt.ylim(0,160)
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("$N(t)$", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15);

The simple idea behind drawing Slope or Direction Fields is that we can calculate slope of the tangent at any point on the graph since it is simply equal to $f(t, y(t))$ in ODE $\frac{dy(t)}{dt}=f(t,y(t))$ or $kN(t)$ in ODE $\frac{dN(t)}{dt}=kN(t)$.  
So we can draw Direction Fields without solving the ODE.

Let's do it for a range of $N(t)$ and $t$ values for $\frac{dN(t)}{dt}=kN(t)$  
  
You'll need to remember some trigonometry to calculate legs from hypotenuse of a right triangle: https://en.wikipedia.org/wiki/Trigonometric_functions

In [ ]:
k = 1
N, t = np.mgrid[-4:4:20j,-4:4:20j]

slope = dNdt = k*N
dt, dN = np.cos(np.arctan(slope)), np.sin(np.arctan(slope))

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14,7), dpi=100, constrained_layout=True)
axs[0].quiver(t, N, dt, dN)
axs[1].streamplot(t, N, dt, dN)

for ax in axs:
    ax.tick_params(labelsize=15)
    ax.grid(which='major', axis='both')
    ax.set_xlabel("Time, $t$",fontsize=20)
    ax.set_ylabel("$N(t)$",fontsize=20)

## Logistic growth

Logistic growth is used to describe the growth of population under conditions that can support max population of size $N_{max}$. The behaviour that we should expect from such a system is that population grows exponentially when $N(t)<<N_{max}$ and declines exponentially when $N(t)>>N_{max}$ and some transition happens when $N(t) \approx N_{max}$. This is the differential expression describing such a system:
  
  $\frac{dN(t)}{dt}=k \cdot N(t) \cdot (1-\frac{N(t)}{N_{max}})$  

  Let's solve this equation using sove_ivp():

In [ ]:
k = 1
N_0 = [1]
tspan = [0, 15]
N_max = 300
def fun(t, N):
    dN_dt = k * N * (1-N/N_max)
    return dN_dt

soln = solve_ivp(fun, tspan, N_0, t_eval=np.linspace(0, 15, 1000))

plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0])
plt.xlabel("Time, $t$", fontsize=15)
plt.ylabel("$N(t)$", fontsize=15)
plt.grid()

Now lets use *quiver()* and *streamplot()* to better understand the behaviour of this system:

In [ ]:
p, t = np.mgrid[0:400:20j,0:15:20j]
slope = 1 * p * (1 - p/300)

dt, dv = np.cos(np.arctan(slope)), np.sin(np.arctan(slope))
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14,7), dpi=100)
axs[0].quiver(t, p, dt, dv, angles="xy")
axs[1].streamplot(t, p, dt, dv)

for ax in axs:
    ax.tick_params(labelsize=15)
    ax.grid(which='major', axis='both')
    ax.set_xlabel("Time (t)",fontsize=20)
    ax.set_ylabel("N(t)",fontsize=20)